In [20]:
import pandas as pd

In [4]:
from dike_model_function import DikeNetwork  # @UnresolvedImport
# Loading in the necessary modules for EMA workbench and functions
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario, ScalarOutcome)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation
from problem_formulation import sum_over, sum_over_time
from ema_workbench import save_results
import copy

# Loading in all the 17 objectives via predefined problem formulation 3
if __name__ == '__main__':
    ema_logging.log_to_stderr(ema_logging.INFO)

    dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
# Replicate the objectives
for outcome in dike_model.outcomes:
    print(repr(type(outcome)))
    

<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>


In [6]:
# Writing a function to create actor specific problem formulations
def problem_formulation_actor(problem_formulation_actor):
   
    # Load the model:
    function = DikeNetwork()
    # workbench model:
    model = Model('dikesnet', function=function)
    # Outcomes are all costs, thus they have to minimized:
    direction = ScalarOutcome.MINIMIZE
    
    model.uncertainties = uncertainties
    model.levers = levers
    
    cost_variables = []
    cost_variables.extend(
    [
        f"{dike}_{e}"
        for e in ["Expected Annual Damage", "Dike Investment Costs"]
        for dike in function.dikelist
    ])
    cost_variables.extend([f"RfR Total Costs"])
    cost_variables.extend([f"Expected Evacuation Costs"])


    if problem_formulation_actor == 4: #RWS
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage',
                            variable_name=['{}_Expected Annual Damage'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction),

            ScalarOutcome('Total Investment Costs',
                            variable_name=['{}_Dike Investment Costs'.format(dike)
                                                for dike in function.dikelist] + ['RfR Total Costs'
                                                                                ] + ['Expected Evacuation Costs'],
                            function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths',
                            variable_name=['{}_Expected Number of Deaths'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction)] 
    
    elif problem_formulation_actor == 5: # GELDERLAND
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A1-4',
                            variable_name=['A.1_Expected Annual Damage' ,'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage'], function=sum_over, kind=direction),

            ScalarOutcome('Investment Costs A1-4',
                            variable_name=['A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs'], function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths in A1-4',
                            variable_name=['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths'], function=sum_over, kind=direction)]
    
    elif problem_formulation_actor == 6: # OVERIJSSEL
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A4', variable_name='A.4_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A5', variable_name='A.5_Expected Annual Damage', function = sum_over, kind=direction),            
            ScalarOutcome('Expected Number of Deaths in A4', variable_name='A.4_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A5', variable_name='A.5_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Total Costs', variable_name=cost_variables, function = sum_over, kind=direction)]
    
    else:
        raise TypeError('unknown identifier')
    return model

In [7]:
# Replicate the uncertainties
uncertainties = dike_model.uncertainties
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [8]:
levers = dike_model.levers 
levers = copy.deepcopy(dike_model.levers)

In [9]:
model = problem_formulation_actor(6)
for outcome in model.outcomes:
    print(repr(type(outcome)))

<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>
<class 'ema_workbench.em_framework.outcomes.ScalarOutcome'>


In [10]:
print(len(model.outcomes))

5


In [11]:
print(sum_over_time(0.0, 0.0, 0.9))  # Expected output: 0.0

0.9


In [12]:
for outcome in model.outcomes:
    print(repr(outcome))

ScalarOutcome('Expected Annual Damage A4', variable_name=('A.4_Expected Annual Damage',), function=<function sum_over at 0x0000020C3C612020>)
ScalarOutcome('Expected Annual Damage A5', variable_name=('A.5_Expected Annual Damage',), function=<function sum_over at 0x0000020C3C612020>)
ScalarOutcome('Expected Number of Deaths in A4', variable_name=('A.4_Expected Number of Deaths',), function=<function sum_over at 0x0000020C3C612020>)
ScalarOutcome('Expected Number of Deaths in A5', variable_name=('A.5_Expected Number of Deaths',), function=<function sum_over at 0x0000020C3C612020>)
ScalarOutcome('Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage', 'A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function sum_ove

In [13]:
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x0000020C3C612020>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x0000020C3C612020>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x0000020C3C612020>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x0000020C3C612020>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x0000020C3C612020>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x0000020C3C612020>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [14]:
reference_values = {
    "Bmax": 175,
    "Brate": 1.5,
    "pfail": 0.5,
    "ID flood wave shape": 4,
    "planning steps": 2,
}
reference_values.update({f"discount rate {n}": 3.5 for n in planning_steps})
refcase_scen = {}

for key in dike_model.uncertainties:
    name_split = key.name.split('_')
    if len(name_split) == 1:

        refcase_scen.update({key.name: reference_values[key.name]})
    else:
        refcase_scen.update({key.name: reference_values[name_split[1]]})
            
ref_scenario = Scenario('reference', **refcase_scen)

In [15]:
# Loading in the right packages for running the optimization
from ema_workbench.em_framework.optimization import (EpsilonProgress)

In [14]:
#convergence_metrics = {EpsilonProgress()}

#with MultiprocessingEvaluator(model,n_processes=-1) as evaluator:
 #   results2 = evaluator.optimize(nfe=5000, searchover='levers',
                                 #convergence=convergence_metrics,
                                 #epsilons=[0.01]*len(model.outcomes), reference=ref_scenario)

#save_results(results2, 'Experiments/Week23_MORDM_Reference_1000_PD6.tar.gz')


[MainProcess/INFO] pool started with 3 workers
5075it [30:16,  2.79it/s]                                                      
[MainProcess/INFO] optimization completed, found 6 solutions
[MainProcess/INFO] terminating pool


In [16]:

y,t = results2

NameError: name 'results2' is not defined

In [18]:
y

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,Expected Annual Damage A4,Expected Annual Damage A5,Expected Number of Deaths in A4,Expected Number of Deaths in A5,Total Costs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.000000e+00,0.000000e+00,0.000000,0.000000,1.448132e+08
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.000000e+00,1.314588e+07,0.000000,0.005199,1.329241e+08
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.000000e+00,1.314588e+07,0.000000,0.014442,1.329219e+08
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4.739687e+06,1.314588e+07,0.002763,0.014442,1.313407e+08
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,4.739687e+06,0.000000e+00,0.002763,0.000000,1.432321e+08
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4.739687e+06,1.314588e+07,0.000995,0.005199,1.313432e+08


In [17]:
y.to_csv('MODRM_DF.csv')

NameError: name 'y' is not defined

### Worst Case

In [18]:
# Worst case specification
worstcase_values ={
    "Bmax": 175,
    "Brate": 1.5,
    "pfail": 0.9,
    "ID flood wave shape": 4,
    "planning steps": 2,
}
worstcase_values.update({f"discount rate {n}": 3.5 for n in planning_steps})

worstcase_scen = {}

for key in dike_model.uncertainties:
    name_split = key.name.split('_')
    if len(name_split) == 1:

        worstcase_scen.update({key.name: worstcase_values[key.name]})
    else:
        worstcase_scen.update({key.name: worstcase_values[name_split[1]]})
            
worst_scenario = Scenario('reference', **worstcase_scen)

In [21]:
#convergence_metrics = {EpsilonProgress()}

#with MultiprocessingEvaluator(model,n_processes=-1) as evaluator:
  #  results3 = evaluator.optimize(nfe=4000, searchover='levers',
     #                            convergence=convergence_metrics,
      #                           epsilons=[0.01]*len(model.outcomes), reference=worst_scenario)

#save_results(results2, 'Experiments/Week23_MORDM_Reference_1000_PD6.tar.gz')


[MainProcess/INFO] pool started with 3 workers

 65%|███████████████████████▌            | 3267/5000 [1:02:42<33:15,  1.15s/it]

 99%|█████████████████████████████████████▍| 3944/4000 [32:23<00:23,  2.41it/s]
4033it [33:11,  2.03it/s]                                                      
[MainProcess/INFO] optimization completed, found 4 solutions
[MainProcess/INFO] terminating pool


In [22]:
worst_case, ui = results3

In [23]:
worst_case.to_csv('MODRM_WC.csv')

In [21]:
import_worst_case = pd.read_csv('MODRM_WC.csv')

In [22]:
# Selecting the policies that are proposed as solutions for further robustness tests
from ema_workbench import Policy

worstcase_policies_to_evaluate = []

for i, policy in import_worst_case.iterrows():
    worstcase_policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [23]:
n_scenarios = 2000
with MultiprocessingEvaluator(model) as evaluator:
    worst_results = evaluator.perform_experiments(n_scenarios,
                                            worstcase_policies_to_evaluate)

[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 2000 scenarios * 4 policies * 1 model(s) = 8000 experiments
100%|██████████████████████████████████████| 8000/8000 [46:07<00:00,  2.89it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [24]:
save_results(worst_results, 'Week23_worst_case_2000_4.tar.gz')

[MainProcess/INFO] results saved successfully to C:\Users\Gebruiker\OneDrive - Delft University of Technology\EPA\Q4 Model based decision making\epa141A_open\final assignment\Week23_worst_case_2000_4.tar.gz


### reference case

In [25]:
reference_case_re = pd.read_csv('MODRM_DF.csv')

In [26]:
# Selecting the policies that are proposed as solutions for further robustness tests
from ema_workbench import Policy

refercase_policies_to_evaluate = []

for i, policy in reference_case_re.iterrows():
    refercase_policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [27]:
n_scenarios = 2000
with MultiprocessingEvaluator(model) as evaluator:
    reference_policies_results = evaluator.perform_experiments(n_scenarios,
                                            refercase_policies_to_evaluate)

[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 2000 scenarios * 6 policies * 1 model(s) = 12000 experiments
100%|██████████████████████████████████| 12000/12000 [1:32:16<00:00,  2.17it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
